In [31]:
import os 
import sys 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json, lxml
#from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
# import folium # map rendering library
from bs4 import BeautifulSoup
warnings.filterwarnings('ignore')

In [21]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
Canada=pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
Canada.to_csv('Toronto.csv')
Toronto_data=pd.read_csv('Toronto.csv')
Toronto_data.head()

,Unnamed: 0,Postal Code,Borough,Neighbourhood
0,0,M1A,Not assigned,Not assigned
1,1,M2A,Not assigned,Not assigned
2,2,M3A,North York,Parkwoods
3,3,M4A,North York,Victoria Village
4,4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [22]:
df=Toronto_data

Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned

In [53]:
df1=df[df['Neighbourhood']!='Not assigned'].reset_index(drop = True)
df1.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


The shape of dataset after removing the Bororough=Not assigned

In [59]:
df1.shape

(103, 3)

In [54]:
print('Number of rows where Neighbourhood is "Not assigned" but borough has value: ', 
      df1[df1['Neighbourhood'] == 'Not assigned'].shape[0])

Number of rows where Neighbourhood is "Not assigned" but borough has value:  0


More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11  in the above table.

In [58]:
postcodes = df1['Postal Code'].values
boroughs = df1['Borough'].values
neighs = df1['Neighbourhood'].values

#create a dictionary with keys as Postcode and Borough, keys of dictioaries are unique
dic = dict({(key1,key2): [] for key1, key2 in zip(postcodes, boroughs)})
print('Number of keys in the dictionary are: ', len(dic.keys()))

#filling the values of keys of dictionary
for postcode, borough, neigh in zip(postcodes,boroughs, neighs):
    key = (postcode, borough)
    dic[key].append(neigh)

df1 = pd.DataFrame(columns = ['Postal Code', 'Borough', 'Neighbourhood'])
for key, value in dic.items():
    postcode, borough, neig = key[0], key[1], value
    neig = ', '.join(neig)
    df1 = df1.append({'Postal Code': postcode,
                     'Borough': borough,
                     'Neighbourhood': neig}, ignore_index = True)
print('Shape of final data is: ', df1.shape)
df1.head(10)

Number of keys in the dictionary are:  103
Shape of final data is:  (103, 3)


,Postal Code,Borough,Neighbourhood
0,M3A,North York,"Parkwoods, Parkwoods"
1,M4A,North York,"Victoria Village, Victoria Village"
2,M5A,Downtown Toronto,"Regent Park, Harbourfront, Regent Park, Harbou..."
3,M6A,North York,"Lawrence Manor, Lawrence Heights, Lawrence Man..."
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government, Q..."
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village, Islin..."
6,M1B,Scarborough,"Malvern, Rouge, Malvern, Rouge"
7,M3B,North York,"Don Mills, Don Mills"
8,M4B,East York,"Parkview Hill, Woodbine Gardens, Parkview Hill..."
9,M5B,Downtown Toronto,"Garden District, Ryerson, Garden District, Rye..."
